This file instantiates objects of the class Particle in the particle.ipynb file, using initial velocities and initial positions generated in the sampling.ipynb file.

Author: 18BME2104 Kaushal

In [4]:
from inspect import getmembers, isfunction, isclass # To inspect imported modules
from IPython.display import display, Math, Latex, clear_output # To use latex typesetting style

import numpy as np # For computations
import sympy as sm # For symbolic use

import pandas as pd # To use large tables
import matplotlib.pyplot as plt #To make plots
import csv # To read csv file of position and velocities required to initialize particles
import import_ipynb
import particle
import field

importing Jupyter notebook from particle.ipynb
Particle describes a particle in the plasma         
 A particle has:         
 mass: m         
 charge: q         
 position: r = [x, y, z]         
 velocity: v = [v_x, v_y, v_z]         
 acceleration: a = [a_x, a_y, a_z]         
importing Jupyter notebook from field.ipynb


In [ ]:
# Probably good idea to make plots, write to csv and other things in another notebook. Let's see.

In [14]:
class Step:
    def __init__(self, file_with_filenames = \
                 '/home/kushik/Kushik/VIT/Eighth semester/MagneticMirror/csvfiles/available files.csv'):
        self.file_with_filenames = file_with_filenames
        self.filenames, self.r_filenames, self.v_filenames = self.read_available_filenames(self.file_with_filenames)
        '''self.filenames would then be a list of availble files to read from
           self.r_filenames would be the list of available files with position written on them
           self.v_filenames would be the list of available files with velocities written on them'''
        self.particles = []
        
    def __string__(self):
        pass
        
    def docstring(self):
        pass
        # print (something)
            
    ### FILE READING SECTION
        
    def read_available_filenames(self):
        rows = []
        filenames = []
        r_filenames = []
        v_filenames = []
        with open(self.file_with_filenames, 'r') as csvfile:
            # creating a csv reader object
            csvreader = csv.reader(csvfile)

            # extracting field names through first row
            #fields = next(csvreader)

            # extracting each data row one by one
            for row in csvreader:
                rows.append(row)
        for row in rows:
            empty = ''
            for char in row:
                empty += char
            filenames.append(empty)
        for filename in filenames:
            r_or_v = filename.split()[1]
            if r_or_v == 'r':
                r_filenames.append(filename)
            else:
                if r_or_v == 'v':
                    v_filenames.append(filename)
                else:
                    raise Exception('There might be some error in filename or how filename is processed in \
                    file reading functions')
    
        return filenames, r_filenames, v_filenames
    
    def print_available_files(self):
        print(self.filenames)
    
    def print_available_r_files(self):
        print(self.r_filenames)
        
    def print_available_v_filenames(self):
        print(self.v_filenames)       
    
    def read_r_or_v_file(self, index):
        '''
        index is the index on the self.filenames list of which the corresponding file is to be read.
        '''
        rows = []
        with open(self.filenames[index], 'r') as csvfile:
            csvreader = csv.reader(csvfile)

            for row in csvreader:
                rows.append(row)
        return row
    
    def read_r_file(self, index):
        '''
        index is the index on the self.r_filenames list of which the corresponding file is to be read.
        '''
        rows = []
        with open(self.r_filenames[index], 'r') as csvfile:
            csvreader = csv.reader(csvfile)

            for row in csvreader:
                rows.append(row)
        return row
    
    def read_v_file(self, index):
        '''
        index is the index on the self.v_filenames list of which the corresponding file is to be read.
        '''
        rows = []
        with open(self.filenames[index], 'r') as csvfile:
            csvreader = csv.reader(csvfile)

            for row in csvreader:
                rows.append(row)
        return row
    
    def reshaper(self, array_from_file):
        three_n = len(array_from_file)
        n = int(three_n / 3)
        reshaped_array = np.array(array_from_file).reshape((n, 3))
        return reshaped_array
        
    def read_r_or_v_file_and_reshape(self):
        array_from_file = self.read_r_or_v_file(index)
        reshaped_array = self.reshaper(array_from_file)
        return reshaped_array
        
    def read_r_file_and_reshape(self):
        array_from_file = self.read_r_file(index)
        reshaped_array = self.reshaper(array_from_file)
        return reshaped_array
    
    def read_v_file_and_reshape(self):
        array_from_file = self.read_v_file(index)
        reshaped_array = self.reshaper(array_from_file)
        return reshaped_array
    
    ### PARTICELS INTIALIZATION SECTION
    def initialize_particles(self, names, q_s, m_s, r_0_s, v_0_s, a_0_s, n):
        '''
        names, q_s, m_s, r_0_s, v_0_s, a_0_s are arrays of name, q, m, r_0, v_0, a_0
        respectively for each particle arranged such that for all of these arrays
        the i_th index represents q, m, r_0, v_0, a_0 of the i_th particle
        n: number of particles initialized, or the length of each of these arrays which is the same
        '''
        
        for i in range(n):
            self.particles.append(Particle(names[i], q_s[i], m_s[i], r_0_s[i], v_0_s[i], a_0_s[i] ))
        ### Might need to adjust for a_0_s[i]
    
    ### FIELDS INITIALIZATION SECTION
    def initialize_fields(self):
        self.fields = Field()
    
    ### TIME STEP SECTION
    def update_particles(self, dt, argsE, argsB):
        for particle in self.particles:
            particle.update(self.fields, dt, argsE, argsB)
    

In [ ]:
#create many particles
# by sampling different positions and velocities using various distribution functions
#Loop
    #get fields at their position
    #move the particles
    
                ''''THIS IN run.ipynb
                '''#create new set of particles after some time, to model flow of particles/ionized gas 
                #in a magnetron sputtering chamber

                #For old particles that move out of the trap bounds (position exceeding some limits), or 
                #modelingf particles being absorbed in the sputtered coating
                #with an associated probability of absorption
                #just delete those particles from the collection
                #(but can be saved in csv files, so that they can stay recorded to be reused later)

In [1]:
print([1,2,3])

[1, 2, 3]


In [1]:
gabs = 'gabs'
gabs[0]

'g'

In [12]:
class Gabs:
    def __init__(self):
        self.habs = 2
    def laps(self):
        self.ll = 3

In [13]:
gabs = Gabs()
gabs.laps()
gabs.ll

3